In [1]:
from time import time
from datetime import datetime,timedelta
import requests
import hashlib
import hmac
import base64
from urllib.parse import urlparse
import pandas as pd #dataframes, easier to use

# For SSL use on Mac
import os
import sys
import certifi
os.environ['REQUESTS_CA_BUNDLE'] = os.path.join(os.path.dirname(sys.argv[0]), certifi.where())


## Use Coinbase API to download price data

In [2]:
#Details
#If either one of the start or end fields are not provided then both fields will be ignored. If a custom time range is not declared then one ending now is selected.
#The granularity field must be one of the following values: {60, 300, 900, 3600, 21600, 86400}. Otherwise, your request will be rejected. These values correspond to 
# #timeslices representing one minute, five minutes, fifteen minutes, one hour, six hours, and one day, respectively.
#If data points are readily available, your response may contain as many as 300 candles and some of those candles may precede your declared start value. 
# #The maximum number of data points for a single request is 300 candles. If your selection of start/end time and granularity will result in more than 300 data points,
# your request will be rejected. If you wish to retrieve fine granularity data over a larger time range, you will need to make multiple requests with new start/end ranges.

#youtube https://www.youtube.com/watch?v=lf92JHVNP0g


#url = "https://api.exchange.coinbase.com/products/BTC-USD/candles?granularity=60&start=1645070640&end=1645069740"

#300min, 5 hours
def getBTCPrice(timeEnd):
    granularity = 60 #60 seconds
    delta = timedelta(minutes=granularity/60)
    #max request size is 300 candles
    timeStart = timeEnd - (300*delta)
    timeStart = timeStart.isoformat()
    timeEnd = timeEnd.isoformat() #must be isoformat

    url="https://api.exchange.coinbase.com/products/{product_id}/candles"
    #https://api-public.sandbox.exchange.coinbase.com/products/BTC-USD/book?level=2
    #live is api.exchange.coinbase.com/products/BTC-USD/book?level=1
    product_id = "BTC-USD"

    requestParam  = {
        "start":timeStart,
        "end":timeEnd,
        "granularity":granularity
    }

    url = url.format(product_id=product_id)
    print("URL: ",url)

    headers={
        "Content-Type":"application/json",
        "ACCEPT":"application/json"
    }

    response = requests.get(url,headers=headers, params=requestParam)

    responseJson = response.json()
    df = pd.DataFrame(response.json(),columns=["timestamp","open","high","low","close","volume"])

    #convert timestamp to a date/time
    df["date"] = pd.to_datetime(df["timestamp"], unit='s')
    df = df[["date","open","high","low","close","volume"]]
    df.set_index("date",inplace=True)
    print(str(timeEnd))
    df.to_csv("BC_BTC_prices.2023.03.04.1min.csv", mode="a",header=False)
    #[timestamp, price_low, price_high, price_open, price_close, volume]

In [ ]:
#request from now, then change the date earlier by 5 hours, n number of times

timeEnd = datetime(2015,10,17,4,58,00)
for i in range(0,1968):    #Use a number divisible by 24. 24 gets 5 full days, such as 1920 or 3840.
    getBTCPrice(timeEnd)
    timeEnd = timeEnd - timedelta(hours=5)

## View data

In [3]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio

In [4]:
# Import data
data = pd.read_csv("BC_BTC_prices.2023.03.04.1min.csv", header=None, index_col=0)

# Rename the index column to "Date"
data = data.rename_axis("Date")

# Add headers
data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

# Reverse the order of rows in data frame, so that earlier times come first in the data.
data = data.iloc[::-1]


In [5]:
# Calculate moving averages
data['4SMA'] = data['Close'].rolling(window=4).mean()
data['16SMA'] = data['Close'].rolling(window=16).mean()
data['64SMA'] = data['Close'].rolling(window=64).mean()

In [6]:
!pip install -U kaleido

In [10]:
# View data
dataview = data.tail(300).copy()
print(dataview.head())

# Chart price data (last 300 prices)
fig = go.Figure(data=[go.Candlestick(x=dataview.index, open=dataview['Open'], high=dataview['High'], low=dataview['Low'], close=dataview['Close'], name='Price')])

# Add moving average lines
fig.add_trace(go.Scatter(x=dataview.index, y=dataview['4SMA'], name='4SMA'))
fig.add_trace(go.Scatter(x=dataview.index, y=dataview['16SMA'], name='16SMA'))
fig.add_trace(go.Scatter(x=dataview.index, y=dataview['64SMA'], name='64SMA'))

fig.update_layout(title='Price and Moving Averages', title_x=0.5)

# Save the chart as an image file
pio.write_image(fig, 'pricechart.png')

                         Open      High       Low     Close    Volume  \
Date                                                                    
2023-03-04 05:00:00  22340.49  22348.21  22347.95  22340.49  1.303631   
2023-03-04 05:01:00  22337.67  22343.80  22340.49  22342.52  2.562976   
2023-03-04 05:02:00  22339.52  22343.18  22341.55  22342.53  3.168018   
2023-03-04 05:03:00  22342.53  22345.50  22342.53  22345.49  1.341763   
2023-03-04 05:04:00  22343.67  22345.41  22345.25  22345.21  1.046043   

                           4SMA         16SMA         64SMA  
Date                                                         
2023-03-04 05:00:00  22340.7925  22336.570000  22347.025156  
2023-03-04 05:01:00  22343.1400  22336.818125  22346.867031  
2023-03-04 05:02:00  22343.4050  22337.220625  22346.673594  
2023-03-04 05:03:00  22342.7575  22337.833750  22346.599219  
2023-03-04 05:04:00  22343.9375  22338.181875  22346.631406  


![My Plot](pricechart.png)